In [1]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../timecorr/'))
from _shared.helpers import isfc, sliding_window
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0.01,0.1,0.5,1]
repetitions=1
noise_num = len(noise_level)
time_range=time_len = block_length * covariance_num
activation_num = 5
subject_num = 5
variance = 1000
activations = np.random.normal(0,1,[noise_num, subject_num, activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[subject_num, activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map,feature_map.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map1,feature_map1.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len):
        cov_temp = (1000-i)*correlation1/float(time_len)+i*correlation2/float(time_len)
        correlations[i] = cov_temp/np.max(abs(cov_temp))
        activations_temp[:, :,i] = np.dot(activations_temp[:,:,i],cholesky(correlations[i]).T)
    for i in range(noise_num):
        activations[i]=activations_temp+np.random.normal(0,noise_level[i],[subject_num, activation_num, time_len])


timecorr_correlations1,timecorr_correlations2 = np.zeros([noise_num,time_len-sliding_window_length+1]),np.zeros([noise_num,time_len-sliding_window_length+1])
sliding_window_correlations1,sliding_window_correlations2 = np.zeros(time_len-sliding_window_length+1),np.zeros(time_len-sliding_window_length+1)

timecorr_MSE1,timecorr_MSE2 = np.zeros([noise_num,time_len-sliding_window_length+1]),np.zeros([noise_num,time_len-sliding_window_length+1])
sliding_window_MSE1,sliding_window_MSE2 = np.zeros(time_len-sliding_window_length+1),np.zeros(time_len-sliding_window_length+1)

timecorr_correlations_single = np.zeros([noise_num,time_range])
sliding_window_correlations_single = np.zeros(time_range)

timecorr_MSE_single = np.zeros([noise_num,time_range])
sliding_window_MSE_single = np.zeros(time_range)
color = ['b','r']

timecorr_recovery = np.zeros([noise_num,time_len,(activation_num * (activation_num-1) / 2)])
print(activations[0].shape)
isfc(activations[0],1000)
for i in range(repetitions):
    cholesky_ramp_correlation_data()
    for v in range(noise_num):
        print(timecorr_recovery.shape,isfc(activations[v],variance).shape)
        timecorr_recovery[v] = isfc(activations[v],variance)
    for timepoint in range(time_len):
        for v in range(noise_num):
            otc1 = pearsonr(timecorr_recovery[v,timepoin], squareform(correlation1,checks=False))[0]
            timecorr_correlations1[v,timepoint] += 0.5 * (log(1+otc1) - log(1-otc1))
            otc2 = pearsonr(timecorr_recovery[v,timepoint], squareform(correlation2,checks=False))[0]
            timecorr_correlations2[v,timepoint] += 0.5 * (log(1+otc2) - log(1-otc2))
            
            timecorr_MSE1[v,timepoint]+=np.mean(np.square(timecorr_recovery[v,timepoint+sliding_window_length/2]-squareform(correlation1,checks=False)))
            timecorr_MSE2[v,timepoint]+=np.mean(np.square(timecorr_recovery[v,timepoint+sliding_window_length/2]-squareform(correlation2,checks=False)))
            
            otcs = pearsonr(timecorr_recovery[v, timepoint+sliding_window_length/2], squareform(correlations[timepoint+sliding_window_length/2],checks=False))[0]
            timecorr_correlations_single[v, timepoint]+= 0.5 * (log(1+otcs) - log(1-otcs))
            
            timecorr_MSE_single[v, timepoint]+=np.mean(np.square(timecorr_recovery[v, timepoint+sliding_window_length/2]-squareform(correlations[timepoint+sliding_window_length/2],checks=False)))
        
        
timecorr_correlations1 /= repetitions
timecorr_correlations1 =  (np.exp(2*timecorr_correlations1) - 1)/(np.exp(2*timecorr_correlations1) + 1)

timecorr_correlations2 /= repetitions
timecorr_correlations2 =  (np.exp(2*timecorr_correlations2) - 1)/(np.exp(2*timecorr_correlations2) + 1)

timecorr_MSE1/= repetitions
timecorr_MSE2/= repetitions

timecorr_correlations_single /= repetitions
timecorr_correlations_single = (np.exp(2*timecorr_correlations_single) - 1)/(np.exp(2*timecorr_correlations_single) + 1) 

timecorr_MSE_single /= repetitions



f, ((ax1,ax2,ax3,ax4),(ax5,ax6,ax7,ax8),(ax9,ax10,ax11,ax12),(ax13,ax14,ax15,ax16)) = plt.subplots(4,4, sharex=True, sharey='row', figsize=(20,20))
a1,a2,a3,a4 = [ax1,ax2,ax3,ax4],[ax5,ax6,ax7,ax8],[ax9,ax10,ax11,ax12],[ax13,ax14,ax15,ax16]
plt.subplots_adjust(top=0.95)
plt.suptitle("ISFC testing with different noise levels",fontsize=20)
a1[0].set_title("noise = 0.01")
a1[1].set_title("noise = 0.1")
a1[2].set_title("noise = 0.5")
a1[3].set_title("noise = 1")
a1[0].set_ylabel("ramping correlation")
a2[0].set_ylabel("ramping MSE")
a3[0].set_ylabel("correlation with ground truth")
a4[0].set_ylabel("MSE with ground truth")
for v in range(noise_num): 
    a1[v].plot(range(time_len),timecorr_correlations1[v],c=color[0],linestyle='-',alpha=1.0)
    a1[v].plot(range(time_len),timecorr_correlations2[v],c=color[1],linestyle='-',alpha=1.0)

    a2[v].plot(range(time_len),timecorr_MSE1[v],c=color[0],linestyle='-',alpha=1.0)
    a2[v].plot(range(time_len),timecorr_MSE2[v],c=color[1],linestyle='-',alpha=1.0)

    a3[v].plot(range(time_len),timecorr_correlations_single[v],c='r',alpha=1,linestyle='-')
    a4[v].plot(range(time_len),timecorr_MSE_single[v],c='r',alpha=1,linestyle='-')
    
plt.show()

(5, 5, 1000)


ValueError: Buffer has wrong number of dimensions (expected 2, got 3)

In [ ]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../timecorr/'))
from _shared.helpers import isfc, sliding_window
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0.01,0.1,0.5,1]
repetitions=1
noise_num = len(noise_level)
time_len = block_length * covariance_num
activation_num = 5
subject_num = 5
variance = 1000
activations = np.random.normal(0,1,[noise_num, subject_num, activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[subject_num, activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map,feature_map.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map1,feature_map1.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len):
        cov_temp = (1000-i)*correlation1/float(time_len)+i*correlation2/float(time_len)
        correlations[i] = cov_temp/np.max(abs(cov_temp))
        activations_temp[:, :,i] = np.dot(activations_temp[:,:,i],cholesky(correlations[i]).T)
    for i in range(noise_num):
        activations[i]=activations_temp+np.random.normal(0,noise_level[i],[subject_num, activation_num, time_len])


timecorr_correlations1,timecorr_correlations2 = np.zeros([noise_num,time_len-sliding_window_length+1]),np.zeros([noise_num,time_len-sliding_window_length+1])
sliding_window_correlations1,sliding_window_correlations2 = np.zeros(time_len-sliding_window_length+1),np.zeros(time_len-sliding_window_length+1)

timecorr_MSE1,timecorr_MSE2 = np.zeros([noise_num,time_len-sliding_window_length+1]),np.zeros([noise_num,time_len-sliding_window_length+1])
sliding_window_MSE1,sliding_window_MSE2 = np.zeros(time_len-sliding_window_length+1),np.zeros(time_len-sliding_window_length+1)

timecorr_correlations_single = np.zeros([noise_num,time_range])
sliding_window_correlations_single = np.zeros(time_range)

timecorr_MSE_single = np.zeros([noise_num,time_range])
sliding_window_MSE_single = np.zeros(time_range)
color = ['b','r']

timecorr_recovery = np.zeros([noise_num,time_len,(activation_num * (activation_num-1) / 2)])

for i in range(repetitions):
    cholesky_ramp_correlation_data()
    for v in range(noise_num):
        timecorr_recovery[v] = isfc(activations[v],variance)
    sliding_window_recovery = sliding_window(activations,sliding_window_length)
    for timepoint in range(time_len-sliding_window_length+1):
        for v in range(noise_num):
            otc1 = pearsonr(timecorr_recovery[v,timepoint+sliding_window_length/2], squareform(correlation1,checks=False))[0]
            timecorr_correlations1[v,timepoint] += 0.5 * (log(1+otc1) - log(1-otc1))
            otc2 = pearsonr(timecorr_recovery[v,timepoint+sliding_window_length/2], squareform(correlation2,checks=False))[0]
            timecorr_correlations2[v,timepoint] += 0.5 * (log(1+otc2) - log(1-otc2))
            
            timecorr_MSE1[v,timepoint]+=np.mean(np.square(timecorr_recovery[v,timepoint+sliding_window_length/2]-squareform(correlation1,checks=False)))
            timecorr_MSE2[v,timepoint]+=np.mean(np.square(timecorr_recovery[v,timepoint+sliding_window_length/2]-squareform(correlation2,checks=False)))
            
            otcs = pearsonr(timecorr_recovery[v, timepoint+sliding_window_length/2], squareform(correlations[timepoint+sliding_window_length/2],checks=False))[0]
            timecorr_correlations_single[v, timepoint]+= 0.5 * (log(1+otcs) - log(1-otcs))
            
            timecorr_MSE_single[v, timepoint]+=np.mean(np.square(timecorr_recovery[v, timepoint+sliding_window_length/2]-squareform(correlations[timepoint+sliding_window_length/2],checks=False)))
        
        sc1 = pearsonr(sliding_window_recovery[timepoint], squareform(correlation1,checks=False))[0]
        sliding_window_correlations1[timepoint] += 0.5 * (log(1+sc1) - log(1-sc1))
        sc2 = pearsonr(sliding_window_recovery[timepoint], squareform(correlation2,checks=False))[0]
        sliding_window_correlations2[timepoint] += 0.5 * (log(1+sc2) - log(1-sc2))
        
        sliding_window_MSE1[timepoint]+=np.mean(np.square(sliding_window_recovery[timepoint]-squareform(correlation1,checks=False)))
        sliding_window_MSE2[timepoint]+=np.mean(np.square(sliding_window_recovery[timepoint]-squareform(correlation2,checks=False)))
         
        swc = pearsonr(sliding_window_recovery[timepoint], squareform(correlations[timepoint+sliding_window_length/2],checks=False))[0]
        sliding_window_correlations_single[timepoint] += 0.5 * (log(1+swc) - log(1-swc))
        
        sliding_window_MSE_single[timepoint]+=np.mean(np.square(sliding_window_recovery[timepoint]-squareform(correlations[timepoint+sliding_window_length/2],checks=False)))        

timecorr_correlations1 /= repetitions
timecorr_correlations1 =  (np.exp(2*timecorr_correlations1) - 1)/(np.exp(2*timecorr_correlations1) + 1)

timecorr_correlations2 /= repetitions
timecorr_correlations2 =  (np.exp(2*timecorr_correlations2) - 1)/(np.exp(2*timecorr_correlations2) + 1)

sliding_window_correlations1 /= repetitions
sliding_window_correlations1 =  (np.exp(2*sliding_window_correlations1) - 1)/(np.exp(2*sliding_window_correlations1) + 1)

sliding_window_correlations2 /= repetitions
sliding_window_correlations2 =  (np.exp(2*sliding_window_correlations2) - 1)/(np.exp(2*sliding_window_correlations2) + 1)

timecorr_MSE1/= repetitions
timecorr_MSE2/= repetitions
sliding_window_MSE1/= repetitions
sliding_window_MSE2/= repetitions

timecorr_correlations_single /= repetitions
timecorr_correlations_single = (np.exp(2*timecorr_correlations_single) - 1)/(np.exp(2*timecorr_correlations_single) + 1) 
sliding_window_correlations_single /= repetitions
sliding_window_correlations_single =(np.exp(2*sliding_window_correlations_single) - 1)/(np.exp(2*sliding_window_correlations_single) + 1) 

timecorr_MSE_single /= repetitions
sliding_window_MSE_single /= repetitions


f, ((ax1,ax2,ax3,ax4),(ax5,ax6,ax7,ax8),(ax9,ax10,ax11,ax12),(ax13,ax14,ax15,ax16)) = plt.subplots(4,4, sharex=True, sharey='row', figsize=(20,20))
a1,a2,a3,a4 = [ax1,ax2,ax3,ax4],[ax5,ax6,ax7,ax8],[ax9,ax10,ax11,ax12],[ax13,ax14,ax15,ax16]
plt.subplots_adjust(top=0.95)
plt.suptitle("timecorr(solid), sliding window(dot)",fontsize=20)
a1[0].set_title("variance = 1000")
a1[1].set_title("variance = 1250")
a1[2].set_title("variance = 1500")
a1[3].set_title("variance = 1750")
a1[0].set_ylabel("ramping correlation")
a2[0].set_ylabel("ramping MSE")
a3[0].set_ylabel("correlation with ground truth")
a4[0].set_ylabel("MSE with ground truth")
for v in range(noise_num): 
    a1[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),timecorr_correlations1[v],c=color[0],linestyle='-',alpha=1.0)
    a1[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),timecorr_correlations2[v],c=color[1],linestyle='-',alpha=1.0)
    a1[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),sliding_window_correlations1,c=color[0],linestyle='--',alpha=0.5)
    ax1.plot(range(sliding_window_length/2,time_len-sliding_window_length/2),sliding_window_correlations2,c=color[1],linestyle='--',alpha=0.5)

    a2[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),timecorr_MSE1[v],c=color[0],linestyle='-',alpha=1.0)
    a2[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),timecorr_MSE2[v],c=color[1],linestyle='-',alpha=1.0)
    a2[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),sliding_window_MSE1,c=color[0],linestyle='--',alpha=0.5)
    a2[v].plot(range(sliding_window_length/2,time_len-sliding_window_length/2),sliding_window_MSE2,c=color[1],linestyle='--',alpha=0.5)


    a3[v].plot(range(lower_limit,upper_limit),timecorr_correlations_single[v],c='r',alpha=1,linestyle='-')
    a3[v].plot(range(lower_limit,upper_limit),sliding_window_correlations_single,c='C1',alpha=0.5,linestyle='--')
    a4[v].plot(range(lower_limit,upper_limit),timecorr_MSE_single[v],c='r',alpha=1,linestyle='-')
    a4[v].plot(range(lower_limit,upper_limit),sliding_window_MSE_single,c='C1',alpha=0.5,linestyle='--')

plt.show()